Primer red neuronal

In [26]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten
from sklearn.metrics import mean_squared_error
import numpy as np

# Carga los datos
users = pd.read_csv('./archive/Users.csv', nrows=20000)
books = pd.read_csv('./archive/Books.csv').sample(n=100000, random_state=42)
ratings = pd.read_csv('./archive/Ratings.csv').sample(n=100000, random_state=42)

def clean_isbn(isbn):
    """Elimina todos los caracteres que no sean números de la cadena ISBN."""
    return re.sub(r'[^0-9]', '', isbn)

books['ISBN'] = books['ISBN'].apply(clean_isbn)

# Usar LabelEncoder para mapear 'ISBN' a valores numéricos únicos
isbn_encoder = LabelEncoder()
books['ISBN'] = isbn_encoder.fit_transform(books['ISBN'])

# Asegúrate de que el valor máximo sea igual o menor que el número de elementos en la capa de Embedding
num_books = books['ISBN'].nunique()  # Número único de libros
max_isbn = books['ISBN'].max()

users['Location'] = users['Location'].astype('category').cat.codes
users['Age'] = users['Age'].astype('category').cat.codes
books['Book-Author'] = books['Book-Author'].astype('category').cat.codes
books['Publisher'] = books['Publisher'].astype('category').cat.codes

# Convierte la columna Year-Of-Publication a tipo int
books['Year-Of-Publication'] = books['Year-Of-Publication'].astype('int')

# Normaliza los datos
users['Age'] = users['Age'] / np.max(users['Age'])
books['Year-Of-Publication'] = books['Year-Of-Publication'] / np.max(books['Year-Of-Publication'])

# Creando la NNRM
book_input = Input(shape=(1,), name='book_input')

# Asegúrate de que 'input_dim' coincida con el número de libros únicos
book_embedding = Embedding(input_dim=num_books, output_dim=128, name='book_embedding')(book_input)

# Aplana las capas de incrustación
book_embedding = Flatten()(book_embedding)

# Crea el modelo
model = Model(inputs=[book_input], outputs=book_embedding)

# Compila el modelo
model.compile(optimizer='adam', loss='mse')

# Entrena el modelo
model.fit([books['ISBN'].values], ratings['Book-Rating'].values, epochs=10)

# Realiza predicciones con el modelo
predicted_ratings = model.predict([books['ISBN'].values])

# Calcula el RMSE
rmse = np.sqrt(mean_squared_error(ratings['Book-Rating'].values, predicted_ratings))
print(f'RMSE: {rmse}')

# Enseñando el accuracy
print(f'Accuracy: {100 - rmse}%')

C:\Users\sebas\AppData\Local\Temp\ipykernel_26580\2816446117.py:12: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('./archive/Books.csv').sample(n=100000, random_state=42)


Epoch 1/10
 234/3125 [=>............................] - ETA: 7:14 - loss: 22.8762